# How to calculate occupancy rate using python
Factor engineering is an important piece in all kinds of analysis. In healthcare analysis, digging more information from administrative data which is designed for billing purposes with limited information is essential. Today I'm going to show you a quick trick to create an occupancy rate from admission and discharge data. It could be data from hospitalization, rehabilitation, or other sources.

***
Suppose you have an inpatient data set looks like the following records and you want to know how many patients stayed in the hospital on a certain date, for example, 1/6/2016.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
df_inpatient = pd.read_csv("inpatient.csv") 
df_inpatient['HOSP_ADMSN_TIME'] = pd.to_datetime(df_inpatient['HOSP_ADMSN_TIME'])
df_inpatient['HOSP_ADMSN_TIME'] = [d.date() for d in df_inpatient['HOSP_ADMSN_TIME']]
df_inpatient['HOSP_DISCH_TIME'] = pd.to_datetime(df_inpatient['HOSP_DISCH_TIME'])
df_inpatient['HOSP_DISCH_TIME'] = [d.date() for d in df_inpatient['HOSP_DISCH_TIME']]
df_inpatient.sort_values('PATIENT_NUM', inplace=True)
df_inpatient.dropna(inplace=True)
df_inpatient.reset_index(drop=True, inplace=True)
df_inpatient

,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,PATIENT_NUM
0,2018-12-06,2018-12-24,5314.0
1,2017-01-04,2017-01-19,13018.0
2,2016-06-16,2016-06-22,17254.0
3,2016-10-05,2016-10-14,17719.0
4,2018-01-31,2018-02-03,19885.0
...,...,...,...
765,2019-02-18,2019-03-18,19621435.0
766,2019-03-28,2019-04-11,20076697.0
767,2019-06-26,2019-07-02,20185096.0
768,2019-05-15,2019-05-21,20185096.0


***
The output data should look like the following.

In [3]:
def func(df_adm, df_disch, df_id): 
    restruc = pd.DataFrame(columns=['Dates','Count']) 
    for i in range(0,df_adm.count()): 
        daterange = pd.date_range(df_adm[i], periods = (df_disch[i]- df_adm[i]).days + 1, freq ='D') 
        df_daterange = daterange.to_frame(index=False, name="Dates")
        df_daterange['Count'] = df_id[i] 
        restruc = restruc.append(df_daterange, ignore_index=True) 
    return restruc.groupby('Dates').count()[['Count']].reset_index()

df_out = func(df_inpatient['HOSP_ADMSN_TIME'],df_inpatient['HOSP_DISCH_TIME'],df_inpatient['PATIENT_NUM'])
df_out.head(10)

,Dates,Count
0,2015-12-29,1
1,2015-12-30,1
2,2015-12-31,1
3,2016-01-01,1
4,2016-01-02,1
5,2016-01-03,1
6,2016-01-04,2
7,2016-01-05,2
8,2016-01-06,2
9,2016-01-07,2


This method has a limitation, that is, if you have 10 beds and a person is discharged while another is admitted on the same day, you might end up with 11 individuals on that day and the occupancy rate would be over 100%.

***
When you have the occupancy rate data, you could plot it and see if there's a seasonal pattern. If there is, then check out seasonal_decompose from statsmodels.tsa.seasonal. If you want to do prediction, check out ARMA from statsmodels.tsa.arima_model.